In [1]:
%matplotlib inline
import sys
import os
import random
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path + "/src/simulations_v2")
from load_params import load_multigroup_params, load_params
from dynamic_population_sim import DynamicPopulationSim
from multi_group_simulation_dynamic_population import MultiGroupSimulationDynamicPopulation

In [2]:
ug_total_pop_size = 13000

# ug_on_campus_pop_size = 6000
# ug_off_campus_pop_size = ug_total_pop_size - ug_on_campus_pop_size # 7000

ug_high_prev_pop_size = 2200
ug_low_prev_pop_size = ug_total_pop_size - ug_high_prev_pop_size # 10800

# ug_high_prev_on_campus_pop_size =  int(ug_high_prev_pop_size * ug_on_campus_pop_size / ug_total_pop_size) # 1015
# ug_high_prev_off_campus_pop_size = ug_high_prev_pop_size - ug_high_prev_on_campus_pop_size # 1185

# ug_low_prev_on_campus_pop_size = int(ug_low_prev_pop_size * ug_on_campus_pop_size / ug_total_pop_size) # 4984
# ug_low_prev_off_campus_pop_size = ug_low_prev_pop_size - ug_low_prev_on_campus_pop_size # 5816

In [3]:
base_params_free = load_params('../src/simulations_v2/params/quarantine_analysis/oct12quarantine/ug_free.yaml')[1]
base_params_selfiso = load_params('../src/simulations_v2/params/quarantine_analysis/oct12quarantine/ug_self_iso.yaml')[1]
base_params_post_movein = load_params('../src/simulations_v2/params/quarantine_analysis/oct12quarantine/ug_post_movein.yaml')[1]

In [4]:
# code copied over from oct_12 notebook just to initialize two dynamic population sim objects
prev_incoming = 16 / 13000 / 0.9
prev_low = prev_incoming / 1.2
prev_high = prev_low * 2 
print('prev:', prev_incoming, prev_low, prev_high)

gateway_testing_fnr = 0.4
fraction_of_immediate_test = 0.73
prev_low_free = prev_low * (fraction_of_immediate_test * gateway_testing_fnr + (1 - fraction_of_immediate_test))
prev_high_free = prev_high * gateway_testing_fnr
print('prev_free:', prev_low_free, prev_high_free)

prev_low_identified_by_gateway = prev_low - prev_low_free
prev_high_identified_by_gateway = prev_high - prev_high_free
print('prev_gateway:', prev_low_identified_by_gateway, prev_high_identified_by_gateway)


movein_free_params = base_params_free.copy()
movein_selfiso_params = base_params_selfiso.copy()
post_movein_params = base_params_post_movein.copy()

movein_selfiso_params['population_size'] = ug_high_prev_pop_size
movein_free_params["initial_ID_prevalence"] = 0
movein_selfiso_params["initial_ID_prevalence"] = st.binom.rvs(ug_high_prev_pop_size, prev_high_free) / ug_high_prev_pop_size
post_movein_params["initial_ID_prevalence"] = 0.5
daily_selfiso_to_free_contacts_pp = 2
daily_free_to_selfiso_contacts_pp = 1

movein_contact_matrix = np.matrix(
    [[movein_free_params['expected_contacts_per_day'], daily_selfiso_to_free_contacts_pp],
    [daily_free_to_selfiso_contacts_pp, movein_selfiso_params['expected_contacts_per_day']]])

movein_time_horizon = 15

free_infectious_low_prev = st.binom.rvs(ug_low_prev_pop_size, prev_low_free)
sampled_days = [random.randint(7, 13) for _ in range(free_infectious_low_prev)]
counts_by_days = dict((t,sampled_days.count(t)) for t in set(sampled_days))
counts_by_days = [counts_by_days[t] if t in counts_by_days else 0 for t in range(7, 14)]
student_returns_by_days = [ug_low_prev_pop_size // 7] * 6 + [ug_low_prev_pop_size - ug_low_prev_pop_size // 7 * 6]

free_group_population_dynamics = {
    t: {'S': student_returns_by_days[i] - counts_by_days[i], 'ID': counts_by_days[i]} 
     for i, t in enumerate(range(7,14))
}
selfiso_group_population_dynamics = {}

dynamic_pop_sim1 = DynamicPopulationSim(
    movein_free_params,
    movein_selfiso_params,
    post_movein_params,
    movein_contact_matrix,
    movein_time_horizon,
    free_group_population_dynamics,
    selfiso_group_population_dynamics
)

dynamic_pop_sim2 = DynamicPopulationSim(
    movein_free_params,
    movein_selfiso_params,
    post_movein_params,
    movein_contact_matrix,
    movein_time_horizon,
    free_group_population_dynamics,
    selfiso_group_population_dynamics
)

prev: 0.0013675213675213675 0.0011396011396011397 0.0022792022792022795
prev_free: 0.0006404558404558406 0.0009116809116809119
prev_gateway: 0.0004991452991452991 0.0013675213675213675


In [5]:
# initialize multigroup dynamic population sim object
contact_matrix = np.matrix([[1,2],[2,1]])
multigroup_sim = MultiGroupSimulationDynamicPopulation([dynamic_pop_sim1, dynamic_pop_sim2], contact_matrix)

In [11]:
# call step() function manually
for _ in range(30):
    multigroup_sim.step()

In [12]:
# record data manually after running simulation
dynamic_pop_sim1.movein_sim.sims[0].sim_df

,S,QS,QI,R,E_0,E_1,E_2,E_3,E_4,E_5,...,SyID_severe_13,SyID_severe_14,SyID_severe_15,SyID_severe_16,SyID_severe_17,SyID_severe_18,SyID_severe_19,cumulative_mild,cumulative_severe,cumulative_outside_infections
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1538,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9,3074,2,1,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,2,0


In [13]:
dynamic_pop_sim2.movein_sim.sims[0].sim_df

,S,QS,QI,R,E_0,E_1,E_2,E_3,E_4,E_5,...,SyID_severe_13,SyID_severe_14,SyID_severe_15,SyID_severe_16,SyID_severe_17,SyID_severe_18,SyID_severe_19,cumulative_mild,cumulative_severe,cumulative_outside_infections
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1539,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,3078,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [14]:
dynamic_pop_sim1.post_movein_sim.sim_df

,S,QS,QI,R,E_0,E_1,E_2,E_3,E_4,E_5,...,SyID_severe_13,SyID_severe_14,SyID_severe_15,SyID_severe_16,SyID_severe_17,SyID_severe_18,SyID_severe_19,cumulative_mild,cumulative_severe,cumulative_outside_infections
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,12941,20,13,10,1,4,0,1,0,0,...,0,0,1,0,0,0,0,18,13,1
2,12939,21,15,13,4,1,0,0,0,0,...,0,1,0,0,0,0,0,21,13,1
3,12932,26,15,15,0,0,1,1,0,0,...,1,0,1,0,0,0,0,22,15,1
4,12932,26,14,17,0,1,1,0,0,0,...,0,1,0,0,0,0,0,22,15,1
5,12933,24,14,20,1,2,0,0,0,0,...,1,0,0,0,0,0,0,23,15,1
6,12926,30,14,22,2,0,0,0,0,0,...,0,0,0,0,0,0,0,24,15,2
7,12926,30,13,25,0,0,0,0,0,0,...,0,0,1,0,0,0,0,25,16,2
8,12921,34,14,27,1,0,0,0,0,0,...,0,1,0,0,0,0,0,26,17,2
9,12918,37,14,29,0,0,0,0,0,0,...,0,0,0,0,0,0,0,27,17,2
